In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [5]:
from sklearn.metrics import mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/08/22 04:46:09 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/22 04:46:09 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1755819227471, experiment_id='1', last_update_time=1755819227471, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
import numpy as np
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(rmse)

7.758715206809494


In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "andres")
    
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)
    print(rmse)

11.167275942353895


In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/usr/local/python/3.12.1/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=2
        )
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=1,
    trials=Trials()
)

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [04:46:12] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.77943                         
[1]	validation-rmse:11.37524                         
[2]	validation-rmse:10.99888                         
[3]	validation-rmse:10.64878                         
[4]	validation-rmse:10.32322                         
[5]	validation-rmse:10.02062                         
[6]	validation-rmse:9.74060                          
[7]	validation-rmse:9.48111                          
[8]	validation-rmse:9.24042                          
[9]	validation-rmse:9.01808                          
[10]	validation-rmse:8.81221                         
[11]	validation-rmse:8.62179                         
[12]	validation-rmse:8.44765                         
[13]	validation-rmse:8.28635                         
[14]	validation-rmse:8.13879                         
[15]	validation-rmse:8.00242                         
[16]	validation-rmse:7.87694                         
[17]	validation-rmse:7.76257                         
[18]	validation-rmse:7.65771

In [29]:
import json

params = {
    'learning_rate': 0.19356366500064653,
    'max_depth': 44,
    'min_child_weight': 1.0019093602514082,
    'objective': 'reg:linear',
    'reg_alpha': 0.019190575638170518,
    'reg_lambda': 0.012312262891608836,
    'seed': 42,
}

with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)

    # Entrenamiento
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    # Predicción y métrica
    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    # === Guardar modelo principal ===
    mlflow.xgboost.log_model(booster, artifact_path="model")

    # === Guardar importancias extra en "model/extra" ===
    importance_types = ["weight", "gain", "cover", "total_gain", "total_cover"]

    for imp_type in importance_types:
        # JSON
        importance = booster.get_score(importance_type=imp_type)
        json_path = f"feature_importance_{imp_type}.json"
        with open(json_path, "w") as f:
            json.dump(importance, f)
        mlflow.log_artifact(json_path, artifact_path="model/extra")

        # PNG
        ax = xgb.plot_importance(booster, importance_type=imp_type)
        fig = ax.figure
        png_path = f"feature_importance_{imp_type}.png"
        fig.savefig(png_path)
        plt.close(fig)
        mlflow.log_artifact(png_path, artifact_path="model/extra")

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [05:21:39] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:10.67753
[1]	validation-rmse:9.53046
[2]	validation-rmse:8.68120
[3]	validation-rmse:8.06867
[4]	validation-rmse:7.62982
[5]	validation-rmse:7.31512
[6]	validation-rmse:7.09094
[7]	validation-rmse:6.93018
[8]	validation-rmse:6.81683
[9]	validation-rmse:6.73580
[10]	validation-rmse:6.67410
[11]	validation-rmse:6.62733
[12]	validation-rmse:6.59347
[13]	validation-rmse:6.56588
[14]	validation-rmse:6.54538
[15]	validation-rmse:6.52699
[16]	validation-rmse:6.51414
[17]	validation-rmse:6.50431
[18]	validation-rmse:6.49483
[19]	validation-rmse:6.48714
[20]	validation-rmse:6.48160
[21]	validation-rmse:6.47721
[22]	validation-rmse:6.47509
[23]	validation-rmse:6.47192
[24]	validation-rmse:6.47051
[25]	validation-rmse:6.46893
[26]	validation-rmse:6.46728
[27]	validation-rmse:6.46483
[28]	validation-rmse:6.46259
[29]	validation-rmse:6.46111
[30]	validation-rmse:6.45950
[31]	validation-rmse:6.45746
[32]	validation-rmse:6.45519
[33]	validation-rmse:6.45258
[34]	validation-rmse:6.

2025/08/22 05:23:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/python/3.12.1/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [05:23:42] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/08/22 05:23:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
